In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
# Import all necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Import all necessary files
iowa_file_train = '/kaggle/input/home-data-for-ml-course/train.csv'
iowa_file_test = '/kaggle/input/home-data-for-ml-course/test.csv'

home_data_train = pd.read_csv(iowa_file_train) # This is all train data
home_data_test = pd.read_csv(iowa_file_test) # This is all test data

y = home_data_train.SalePrice # This is what we want to predict
home_data_train.drop(['SalePrice', 'Id'], axis=1, inplace=True ) # This is all the rest (all features) without 'Id' in train data
home_data_test.drop(['Id'], axis=1, inplace=True ) # This is all the rest (all features) without 'Id' in test data
X_train = home_data_train # For shortly access
X_test = home_data_test

In [3]:
# Find columns with missing
cols_w_m_n_train = [col for col in X_train.columns if X_train[col].isnull().any()] #Missing in train
cols_w_m_n_test = [col for col in X_test.columns if X_test[col].isnull().any()] #Missing in test


print(cols_w_m_n_train, '\n',cols_w_m_n_test)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'] 
 ['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']


In [4]:
# Find all features that have categorical data
s = (X_train.dtypes == 'object')
cols_s = list(s[s].index)

ss = (X_test.dtypes == 'object')
cols_ss = list(s[s].index)

print("Categorical variables in train data:", cols_s)
print("Categorical variables in test data:", cols_ss)

Categorical variables in train data: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
Categorical variables in test data: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'

It's ok, we have the same amount of categorical data in train and test both.

In [5]:
n_unique_train = list(map(lambda col: X_train[col].nunique(), cols_s))
d = dict(zip(cols_s, n_unique_train))
sorted(d.items(), key = lambda x: x[1])

[('Street', 2),
 ('Alley', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('GarageFinish', 3),
 ('PavedDrive', 3),
 ('PoolQC', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('MasVnrType', 4),
 ('ExterQual', 4),
 ('BsmtQual', 4),
 ('BsmtCond', 4),
 ('BsmtExposure', 4),
 ('KitchenQual', 4),
 ('Fence', 4),
 ('MiscFeature', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Electrical', 5),
 ('FireplaceQu', 5),
 ('GarageQual', 5),
 ('GarageCond', 5),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('BsmtFinType1', 6),
 ('BsmtFinType2', 6),
 ('Heating', 6),
 ('GarageType', 6),
 ('SaleCondition', 6),
 ('Functional', 7),
 ('Condition2', 8),
 ('HouseStyle', 8),
 ('RoofMatl', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [6]:
n_unique_test = list(map(lambda col: X_test[col].nunique(), cols_ss))
dd = dict(zip(cols_ss, n_unique_test))
sorted(dd.items(), key = lambda x: x[1])

[('Utilities', 1),
 ('Street', 2),
 ('Alley', 2),
 ('CentralAir', 2),
 ('PoolQC', 2),
 ('LandSlope', 3),
 ('GarageFinish', 3),
 ('PavedDrive', 3),
 ('MiscFeature', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('RoofMatl', 4),
 ('MasVnrType', 4),
 ('ExterQual', 4),
 ('BsmtQual', 4),
 ('BsmtCond', 4),
 ('BsmtExposure', 4),
 ('Heating', 4),
 ('Electrical', 4),
 ('KitchenQual', 4),
 ('GarageQual', 4),
 ('Fence', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('Condition2', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('FireplaceQu', 5),
 ('GarageCond', 5),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('BsmtFinType1', 6),
 ('BsmtFinType2', 6),
 ('GarageType', 6),
 ('SaleCondition', 6),
 ('HouseStyle', 7),
 ('Functional', 7),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 13),
 ('Exterior2nd', 15),
 ('Neighborhood', 25)]

In [7]:
#First of all let's impute the missing values.

X_train = X_train.fillna(value = {'Alley': 'N','MasVnrType': 'N','BsmtQual': 'TA','BsmtCond': 'TA', 'BsmtExposure': 'No','BsmtFinType1': 'N', 'BsmtFinType2': 'N', 
'Electrical': 'SBrkr','FireplaceQu': 'N', 'GarageType': 'N', 'GarageFinish': 'N','GarageQual': 'N', 'GarageCond': 'N', 'PoolQC': 'N', 'Fence': 'N', 'MiscFeature': 'N', 
'LotFrontage': X_train['LotFrontage'].mean(), 'MasVnrArea': 0, 'GarageYrBlt': X_train['GarageYrBlt'].mean()})

X_test = X_test.fillna(value = {'LotFrontage': X_test['LotFrontage'].mean(), 'MasVnrArea': 0, 'GarageYrBlt': X_test['GarageYrBlt'].mean(), #Test DATA
'BsmtFinSF1' : X_test['LotFrontage'].mean(),'BsmtFinSF2' : X_test['LotFrontage'].mean(),'BsmtUnfSF' : X_test['LotFrontage'].mean(), 'TotalBsmtSF' : X_test['LotFrontage'].mean(),       
'BsmtFullBath' : X_test['LotFrontage'].median(),'BsmtHalfBath' : X_test['LotFrontage'].median(),'GarageCars' : X_test['LotFrontage'].median(),'GarageArea' : X_test['LotFrontage'].mean(),
'MSZoning': 'RL','Alley': 'N','Utilities': 'N','Exterior1st': 'N','Exterior2nd': 'N','MasVnrType': 'N','BsmtQual': 'TA','BsmtCond': 'TA','BsmtExposure': 'No','BsmtFinType1': 'N',
'BsmtFinType2': 'N','KitchenQual': 'TA','Functional': 'Typ','FireplaceQu': 'N','GarageType': 'N','GarageFinish': 'N','GarageQual': 'N','GarageCond': 'N','PoolQC': 'N',
'Fence': 'N','MiscFeature': 'N','SaleType': 'WD',})

In [8]:
X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,N,Reg,Lvl,AllPub,Inside,...,0,0,N,N,N,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,N,Reg,Lvl,AllPub,FR2,...,0,0,N,N,N,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,N,IR1,Lvl,AllPub,Inside,...,0,0,N,N,N,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,N,IR1,Lvl,AllPub,Corner,...,0,0,N,N,N,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,N,IR1,Lvl,AllPub,FR2,...,0,0,N,N,N,0,12,2008,WD,Normal


In [9]:
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,N,Reg,Lvl,AllPub,Inside,...,120,0,N,MnPrv,N,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,N,IR1,Lvl,AllPub,Corner,...,0,0,N,N,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,N,IR1,Lvl,AllPub,Inside,...,0,0,N,MnPrv,N,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,N,IR1,Lvl,AllPub,Inside,...,0,0,N,N,N,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,N,IR1,HLS,AllPub,Inside,...,144,0,N,N,N,0,1,2010,WD,Normal


In [10]:
#Secondly, we need to label features, that: categorical, have cardinality 2 or more than 9. As we found earlier it is:
#in train [('Street', 2), ('Alley', 2), ('Utilities', 2), ('CentralAir', 2), ('Exterior1st', 15), ('Exterior2nd', 16), ('Neighborhood', 25)]
#in test [('Street', 2), ('Alley', 2), ('Utilities', 2), ('CentralAir', 2), ('Exterior1st', 15), ('Exterior2nd', 16), ('Neighborhood', 25)] the same.

#Make copy to avoid changing original data
label_X_train = X_train.copy()
label_X_test = X_test.copy()

#Apply label encoding to each column with a such data:
label_encoder = LabelEncoder()
for col in ['Street','Alley','Utilities','CentralAir','Exterior1st','Exterior2nd','Neighborhood','Utilities','PoolQC','MiscFeature',
            'Electrical','GarageQual','Heating','Condition2','HouseStyle','RoofMatl','Exterior1st','Exterior2nd','MSZoning','KitchenQual','Functional', 'SaleType']:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_test[col] = label_encoder.fit_transform(X_test[col])
    
#As it would found later, we got different cardinaluty in train and test data in the following columns, 
#among those wich we would like to do with OH_encoding:
#[('GarageFinish', 3),('MasVnrType', 4),('Fence', 4),('Electrical', 5),('FireplaceQu', 5),('GarageCond', 5),('BsmtFinType1', 6),
#('BsmtFinType2', 6),('Heating', 6),('GarageType', 6),('Condition2', 8),('HouseStyle', 8),('RoofMatl', 8). 
#So, we need to label them here too.


In [11]:
label_X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3,65.0,8450,1,1,Reg,Lvl,0,Inside,...,0,0,3,N,1,0,2,2008,8,Normal
1,20,3,80.0,9600,1,1,Reg,Lvl,0,FR2,...,0,0,3,N,1,0,5,2007,8,Normal
2,60,3,68.0,11250,1,1,IR1,Lvl,0,Inside,...,0,0,3,N,1,0,9,2008,8,Normal
3,70,3,60.0,9550,1,1,IR1,Lvl,0,Corner,...,0,0,3,N,1,0,2,2006,8,Abnorml
4,60,3,84.0,14260,1,1,IR1,Lvl,0,FR2,...,0,0,3,N,1,0,12,2008,8,Normal


In [12]:
label_X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,2,80.0,11622,1,1,Reg,Lvl,0,Inside,...,120,0,2,MnPrv,1,0,6,2010,8,Normal
1,20,3,81.0,14267,1,1,IR1,Lvl,0,Corner,...,0,0,2,N,0,12500,6,2010,8,Normal
2,60,3,74.0,13830,1,1,IR1,Lvl,0,Inside,...,0,0,2,MnPrv,1,0,3,2010,8,Normal
3,60,3,78.0,9978,1,1,IR1,Lvl,0,Inside,...,0,0,2,N,1,0,6,2010,8,Normal
4,120,3,43.0,5005,1,1,IR1,HLS,0,Inside,...,144,0,2,N,1,0,1,2010,8,Normal


In [13]:
#Thirdly, we'll use OH_encoding but just with get dummies, now, for all date, because we do what we wanted before, 
#and now there are just categorical features that must be divided by one-hot encoding. 

label_X_train = pd.get_dummies(label_X_train)
label_X_test = pd.get_dummies(label_X_test)

In [14]:
label_X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition2,HouseStyle,...,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_N,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,3,65.0,8450,1,1,0,5,2,5,...,0,0,0,1,0,0,0,0,1,0
1,20,3,80.0,9600,1,1,0,24,2,2,...,0,0,0,1,0,0,0,0,1,0
2,60,3,68.0,11250,1,1,0,5,2,5,...,0,0,0,1,0,0,0,0,1,0
3,70,3,60.0,9550,1,1,0,6,2,5,...,0,0,0,1,1,0,0,0,0,0
4,60,3,84.0,14260,1,1,0,15,2,5,...,0,0,0,1,0,0,0,0,1,0


In [15]:
label_X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition2,HouseStyle,...,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_N,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,2,80.0,11622,1,1,0,12,2,2,...,0,1,0,0,0,0,0,0,1,0
1,20,3,81.0,14267,1,1,0,12,2,2,...,0,0,0,1,0,0,0,0,1,0
2,60,3,74.0,13830,1,1,0,8,2,4,...,0,1,0,0,0,0,0,0,1,0
3,60,3,78.0,9978,1,1,0,8,2,4,...,0,0,0,1,0,0,0,0,1,0
4,120,3,43.0,5005,1,1,0,22,2,2,...,0,0,0,1,0,0,0,0,1,0


So, we got the following result:
in train data there are 242 columns
in test data there are 228 columns

The model wouldn't accept it. Well, why we get different amount of columns? There were following columns, that get_dummies:
1. ('Alley', 3),
2. ('LandSlope', 3),
3. ('PavedDrive', 3),
4. ('PoolQC', 3),
5. ('LotShape', 4),
6. ('LandContour', 4),
7. ('RoofMatl', 4),
8. ('ExterQual', 4),
9. ('BsmtQual', 4),
10. ('BsmtCond', 4),
11. ('BsmtExposure', 4),
12. ('Heating', 4),
13. ('Electrical', 4),
14. ('KitchenQual', 4),
15. ('GarageFinish', 4),
16. ('MiscFeature', 4),
17. ('MSZoning', 5),
18. ('LotConfig', 5),
19. ('Condition2', 5),
20. ('BldgType', 5),
21. ('MasVnrType', 5),
22. ('ExterCond', 5),
23. ('HeatingQC', 5),
24. ('GarageQual', 5),
25. ('Fence', 5),
26. ('RoofStyle', 6),
27. ('Foundation', 6),
28. ('FireplaceQu', 6),
29. ('GarageCond', 6),
30. ('SaleCondition', 6),
31. ('HouseStyle', 7),
32. ('BsmtFinType1', 7),
33. ('BsmtFinType2', 7),
34. ('Functional', 7),
35. ('GarageType', 7),
36. ('Condition1', 9),
37. ('SaleType', 9).

And we have 242 - 228 = 14 different columns, what is it? Well, there are at least 2 problems:
1. Some columns has different cardinality in train and test.
2. Some of these columns have NONE in test and have no missing values in train and vise versa.
What can we do?
1. We can find these columns and drop them,
2. or just label them too, like others.

Ok, done. Now we have in train 173 columns and 170 in test data, still why?
That's because there are some NONE that were inpute like 'N' in train data, and in test data there were no empty (in such columns).

So, finally we get in both data 179 columns with following label features:
['Street','Alley','Utilities','CentralAir','Exterior1st','Exterior2nd','Neighborhood','Utilities','PoolQC','MiscFeature',
'Electrical','GarageQual','Heating','Condition2','HouseStyle','RoofMatl','Exterior1st','Exterior2nd','MSZoning','KitchenQual','Functional', 'SaleType']

In [16]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

RFR = RandomForestRegressor(n_estimators = 900, random_state=0) 
accuracies_RFR = -1*cross_val_score(estimator= RFR ,X= label_X_train, y=y, cv=5, scoring = 'neg_mean_absolute_error')
print('All scoring_RFR:', accuracies_RFR ,'\n', 'Mean score_RFR:', accuracies_RFR.mean())

All scoring_RFR: [17669.36334475 17415.82134703 17527.04043379 16111.39246575
 18759.69039574] 
 Mean score_RFR: 17496.66159741248


In [17]:
XGB = XGBRegressor(n_estimators = 10000, learning_rate = 0.095)
XGB.fit(label_X_train, y, early_stopping_rounds = 55, eval_set = [(label_X_train, y)], verbose = False)
accuracies_XGB = -1*cross_val_score(estimator= XGB ,X = label_X_train, y=y, cv=5, scoring = 'neg_mean_absolute_error')
print('All scoring_XGB:', accuracies_XGB ,'\n', 'Mean score_XGB:', accuracies_XGB.mean())

All scoring_XGB: [15873.18126605 16344.80297517 17464.51700289 14491.98355897
 16910.60231164] 
 Mean score_XGB: 16217.017422945204


In [18]:
from xgboost import XGBRegressor
# Testing the model
# Split the data
s_X_train, s_X_test, s_y_train, s_y_test = train_test_split(label_X_train, y, train_size=0.80, test_size=0.20, random_state=0)

# Now, let's test our model!
model = XGBRegressor(n_estimators = 10000, learning_rate = 0.095)
model.fit(s_X_train, s_y_train, early_stopping_rounds = 55, eval_set = [(s_X_test, s_y_test)], verbose = False)

val_predictions = model.predict(s_X_test)
val_mae = mean_absolute_error(val_predictions, s_y_test)
print(val_mae)


16508.58566994863


In [19]:
# Now, let's submit our model!


#model.fit(label_X_train, y)
test_preds = XGB.predict(label_X_test)

output = pd.DataFrame({'Id': pd.read_csv(iowa_file_test).Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)